# 1. XLang v1 — Minimal Excel Compiler Notebook

**Project:** Output Representation Optimisation (ORO)  
**Version:** 1.0 (Preview)  
**Date:** 2025-11-17  
**Author:** Jackson Chai

---

## 1.1 Purpose of This Notebook

This notebook implements a minimal working version of XLang v1, a concise markup language designed for large language models (LLMs) to generate Excel workbooks directly.

The main objectives are to:

1. Understand the basic structure of XLang.  
2. Build a minimal compiler in Python.  
3. Test the compilation end to end using a simple example.  
4. Provide a foundation for extending XLang with styles, merges, sequences and patterns.

---

## 1.2 Scope of This Version

In this first version of the notebook, we only support the core tags:

1. xworkbook  
2. xsheet  
3. xrow  
4. xv  
5. xcell  

Support for additional tags such as xstyle, xmerge, xseq, xplace, xrepeat, xpattern and xfill will be added later, after we verify that the core pipeline works correctly.


In [4]:
# ============================================================
# 1. Imports and basic setup
# ============================================================

# Uncomment the following line if your environment does not have openpyxl installed.
# %pip install openpyxl

from xml.etree import ElementTree as ET
from openpyxl import Workbook, load_workbook
from pathlib import Path

# 2. Column Letter Conversion Helper

## 2.1 Motivation

Excel columns are labelled using letters such as A, B, C, …, Z, AA, AB and so on.  
To place values programmatically, we must convert these letters into numeric indices.

Examples:

- A becomes 1  
- B becomes 2  
- Z becomes 26  
- AA becomes 27  

The following helper function handles this conversion in a robust way.


In [5]:
# ============================================================
# 2. Column letter to index helper
# ============================================================

def col_letter_to_index(col: str) -> int:
    """
    Convert Excel column letters (A, B, Z, AA, AB etc.)
    into a 1-based integer column index.
    """
    col = col.strip().upper()
    result = 0

    for ch in col:
        if not ("A" <= ch <= "Z"):
            raise ValueError(f"Invalid column letter: {col}")
        result = result * 26 + (ord(ch) - ord("A") + 1)

    return result

# 3. Minimal XLang Validator

## 3.1 Why Validate

Before compiling XLang into Excel, it is useful to catch simple structural errors early.  
A minimal validator helps ensure that the core tags are used correctly and prevents confusing runtime failures.

## 3.2 Validation Rules Implemented

For this initial version, the validator checks the following:

1. The root tag must be `xworkbook`.  
2. Each `xsheet` must have a `name` attribute.  
3. Each `xrow` must have an `r` attribute, indicating the row index.  
4. Each `xcell` must have both `addr` (address) and `v` (value) attributes.

Advanced validation for styles, merges and sequences will be introduced later.


In [ ]:
# ============================================================
# 3. Minimal XLang validator (with type hint checking)
# ============================================================

ALLOWED_TYPES = {"number", "string", "date", "bool"}

def validate_xlang_minimal(root: ET.Element) -> list[str]:
    """
    Perform minimal validation of an XLang document.

    Checks:
      - Root tag is xworkbook
      - xsheet has name
      - xrow has r
      - xcell has addr and v
      - Optional t attributes are from an allowed set
    """
    errors: list[str] = []

    # 3.2.1 Check root tag
    if root.tag != "xworkbook":
        errors.append(f"Root tag must be 'xworkbook' but found '{root.tag}'")
        return errors

    for sheet in root.findall("xsheet"):
        # xsheet name is required
        if "name" not in sheet.attrib:
            errors.append("xsheet missing required attribute 'name'")

        # xrow must have a row index
        for xrow in sheet.findall("xrow"):
            if "r" not in xrow.attrib:
                errors.append("xrow missing required attribute 'r'")

        # xcell must specify addr and v; validate optional t
        for xcell in sheet.findall("xcell"):
            if "addr" not in xcell.attrib:
                errors.append("xcell missing required attribute 'addr'")
            if "v" not in xcell.attrib:
                errors.append("xcell missing required attribute 'v'")
            t = xcell.attrib.get("t")
            if t is not None and t not in ALLOWED_TYPES:
                errors.append(
                    f"xcell at {xcell.attrib.get('addr', '?')} has invalid type hint t='{t}'"
                )

    return errors


# 4. Minimal XLang Compiler

## 4.1 Supported Tags in This Version

The compiler implemented here supports the following tags:

1. xworkbook  
2. xsheet  
3. xrow  
4. xv  
5. xcell  

This is enough to express:

- Basic headers and data rows  
- Single-cell labels  
- Simple formulas  

## 4.2 Compilation Steps

The compilation pipeline runs as follows:

1. Parse the XLang text into an XML tree using ElementTree.  
2. Validate the basic structure with the minimal validator.  
3. Create a new Excel workbook using openpyxl.  
4. For each `xsheet`:
   1. Create a worksheet with the given name.  
   2. For each `xrow`:
      - Determine the row index and starting column.  
      - Write each `xv` value into successive columns.  
   3. For each `xcell`:
      - Write the value directly into the specified address.  
5. Save the workbook to the given path as a `.xlsx` file.

Future extensions will add support for more XLang tags and behaviours.


In [ ]:
# ============================================================
# 4. Minimal XLang compiler implementation (with type inference)
# ============================================================

# ============================================================
# 4.1 Value inference helper
# ============================================================

def infer_value(raw: str, type_hint: str | None = None):
    """
    Infer the correct Python type for a cell value.

    Rules:
      - If raw starts with '=', always treat as a formula string.
      - If type_hint is provided:
          - 'number'  -> try int, then float
          - 'string'  -> keep as string
          - 'date'    -> leave as string for now (date parsing can be added later)
          - 'bool'    -> parse 'TRUE'/'FALSE' etc.
      - If no type_hint:
          - Try to parse as integer or float if it looks numeric.
          - Otherwise keep as string.
    """
    if raw is None:
        return None

    raw = str(raw)

    # Formulas must be kept as strings starting with '='
    if raw.startswith("="):
        return raw

    # Honour explicit type hint if present
    if type_hint == "string":
        return raw

    if type_hint == "bool":
        upper = raw.strip().upper()
        if upper in {"TRUE", "YES"}:
            return True
        if upper in {"FALSE", "NO"}:
            return False
        # Fall back to raw string if ambiguous
        return raw

    if type_hint == "number":
        # Try integer first, then float
        try:
            return int(raw)
        except ValueError:
            try:
                return float(raw)
            except ValueError:
                return raw  # fallback

    if type_hint == "date":
        # For now, do not parse; keep as string (you can add proper date parsing later)
        return raw

    # No type_hint: try to auto-detect number
    stripped = raw.strip()
    # Simple numeric detection: optional sign, digits, optional decimal
    import re
    if re.fullmatch(r"[+-]?\d+", stripped):
        try:
            return int(stripped)
        except ValueError:
            pass
    if re.fullmatch(r"[+-]?\d*\.\d+", stripped):
        try:
            return float(stripped)
        except ValueError:
            pass

    # Fallback: treat as plain string
    return raw
# ============================================================

def compile_xlang_to_xlsx(xlang_text: str, output_path: str | Path) -> None:
    """
    Compile a minimal subset of XLang into an Excel .xlsx file.

    Supported tags:
        - xworkbook
        - xsheet
        - xrow
        - xv
        - xcell

    Values are automatically cast:
        - Formulas: kept as strings starting with '='
        - Numeric-looking strings: converted to int or float
        - Everything else: left as string
        - Optional t attribute on xcell can force a type
    """
    # 4.2.1 Parse the XLang input
    root = ET.fromstring(xlang_text)

    # 4.2.2 Validate basic structure
    errors = validate_xlang_minimal(root)
    if errors:
        formatted = "\n".join("  - " + e for e in errors)
        raise ValueError("Invalid XLang:\n" + formatted)

    # 4.2.3 Create a new workbook and remove the default blank sheet
    wb = Workbook()
    wb.remove(wb.active)

    # 4.2.4 Process each sheet
    for xsheet in root.findall("xsheet"):
        sheet_name = xsheet.attrib["name"]
        ws = wb.create_sheet(title=sheet_name)

        # 4.2.4.1 Process xrow elements
        for xrow in xsheet.findall("xrow"):
            row_idx = int(xrow.attrib["r"])
            start_col_letter = xrow.attrib.get("c", "A")
            start_col_idx = col_letter_to_index(start_col_letter)

            for offset, xv in enumerate(xrow.findall("xv")):
                raw_value = xv.text or ""
                value = infer_value(raw_value, None)
                ws.cell(
                    row=row_idx,
                    column=start_col_idx + offset,
                    value=value
                )

        # 4.2.4.2 Process xcell elements (direct addressing)
        for xcell in xsheet.findall("xcell"):
            addr = xcell.attrib["addr"]
            raw_value = xcell.attrib["v"]
            type_hint = xcell.attrib.get("t")
            value = infer_value(raw_value, type_hint)
            ws[addr] = value

    # 4.2.5 Ensure parent directory exists and save workbook
    output_path = Path(output_path)
    output_path.parent.mkdir(parents=True, exist_ok=True)
    wb.save(str(output_path))


# 5. Example XLang Document

## 5.1 KPI Example

This example describes a very simple worksheet named `KPI`:

1. A header row with `Region` and `Sales`.  
2. Two rows of data: `North`, `South` with numeric values.  
3. A `Total` label.  
4. A `SUM` formula that adds the sales values.

We will compile this XLang text into `kpi_example.xlsx`.


In [8]:
# ============================================================
# 5. Example XLang: KPI sheet
# ============================================================

kpi_example_xlang = """
<xworkbook>
  <xsheet name="KPI">
    <xrow r="1"><xv>Region</xv><xv>Sales</xv></xrow>
    <xrow r="2"><xv>North</xv><xv>120000</xv></xrow>
    <xrow r="3"><xv>South</xv><xv>98000</xv></xrow>
    <xcell addr="A4" v="Total"/>
    <xcell addr="B4" v="=SUM(B2:B3)"/>
  </xsheet>
</xworkbook>
""".strip()


# 6. Compile the Example Workbook

## 6.1 Compilation

The cell below compiles the example XLang text into an Excel file named `kpi_example.xlsx`.

After you run it:

1. Confirm that the file is created in your working directory.  
2. Open it in Excel to check that:
   - The sheet is called `KPI`.  
   - Cell A1 contains `Region`.  
   - Cell B1 contains `Sales`.  
   - Cell B4 contains the formula `=SUM(B2:B3)`.


In [ ]:
# ============================================================
# 6. Compile the example XLang into Excel (output/ folder)
# ============================================================

output_dir = Path("output")
output_dir.mkdir(parents=True, exist_ok=True)

output_file = output_dir / "kpi_example.xlsx"

compile_xlang_to_xlsx(kpi_example_xlang, output_file)

print(f"Workbook written to: {output_file.resolve()}")


Workbook written to: C:\Users\Jackson.chai\Research\xlang\src\notebook\kpi_example.xlsx


# 7. Programmatic Verification

## 7.1 Loading the Generated Workbook

To be confident in the compiler, it is useful to verify the contents of the generated workbook inside Python.

We expect the following:

1. A1 should be `Region`.  
2. B1 should be `Sales`.  
3. A2 should be `North`.  
4. B2 should be `120000`.  
5. A4 should be `Total`.  
6. B4 should contain the formula `=SUM(B2:B3)`.

The next cell loads the workbook with `data_only=False` so that formulas are preserved rather than evaluated.


In [ ]:
# ============================================================
# 7. Inspect the generated workbook for correctness
# ============================================================

wb = load_workbook(output_file, data_only=False)
ws = wb["KPI"]

print("A1:", ws["A1"].value)
print("B1:", ws["B1"].value)
print("A2:", ws["A2"].value)
print("B2:", ws["B2"].value, type(ws["B2"].value))
print("A4:", ws["A4"].value)
print("B4 formula:", ws["B4"].value)
print("B4 data type:", ws["B4"].data_type)


A1: Region
B1: Sales
A2: North
B2: 120000
A4: Total
B4 formula: =SUM(B2:B3)
B4 data type: f
